# 1. Find the address of your FLM in your LAN

In [ ]:
from zeroconf_discover import discover_flm

In [ ]:
flm = discover_flm()

In [ ]:
flm

# 2. Get your Water Sensor configuration

In [ ]:
from mqtt_discover import discover_sensors_by_type

In [ ]:
water_sensors = discover_sensors_by_type(flm, type='water')

In [ ]:
water_sensors

In [ ]:
sensor_config = water_sensors[0]

# 3. Create logic

## Callback function

We want to find a water leak, by exploiting the fact that the water flow always has to go to zero at some point. If it doesn't, it means that water is constantly flowing and you have a problem.

Since water meters are pulse counters, we are simply going define a timespan wherein no pulses should arrive, and a reporting period wherein this timespan should lie.

For instance: I excpect that in a period of 24 hours, there will be at least 1 hour without pulses. If that hour is not found, we detect a leak.

Since such a method requires that some state is persisted, a simple function does not suffice. We will need to subclass the `SensorHandler`.

In [ ]:
from pandas import Timestamp
from handlers import SensorHandler

class WaterLeakHandler(SensorHandler):
    def __init__(self, timespan=3600, reporting_period=86400, **kwargs):
        self.timespan = timespan
        self.reporting_period = reporting_period
        self.last_pulse = 0
        self.last_gap = 0
        
        super(WaterLeakHandler, self).__init__(**kwargs)
        
        self.on_message = self.detect_leak
        
    def detect_leak(self, timestamp, **kwargs):
        detected = self._detect_leak(timestamp=timestamp, **kwargs)
        self.last_pulse = timestamp
        return detected
        
    def _detect_leak(self, timestamp, **kwargs):
        # if the time between this and last pulse is larger than `timespan`, we have a gap and we can save it
        if timestamp - self.last_pulse > self.timespan:
            self.last_gap = timestamp
        else:
            # if not, we go and see if the last gap was longer ago then the reporting period
            if timestamp - self.last_gap > self.reporting_period:
                print("Leak detected! Haven't had {:.2f} minutes without water consumption since {}".format(self.timespan/60, str(Timestamp.fromtimestamp(self.last_gap))))
                return True
        return False

## Create Handler

In [ ]:
# configure timespan and reporting period to something low, so you can turn on your tap and see if it works
handler = WaterLeakHandler(sconfig=sensor_config, timespan=5, reporting_period=10)

# 4. Create an MQTT subscriber, and run!

In [ ]:
from mqtt_subscriber import Subscriber

In [ ]:
subscriber = Subscriber(flm=flm, handlers=[handler])

In [ ]:
subscriber.run()